In [101]:
#Import most required functions, defines 'pretty' function for txt file, and holds client key/secret
import json 
import csv
import requests_oauthlib
import unittest 
import webbrowser 
import requests 



def pretty(obj):
    return json.dumps(obj, sort_keys=True, indent=2)

client_key = '3bvorYE03pVNDoBAQn1OOyHPy'
client_secret = 'BkDRZVyZINMlBXjVCrzElOJTomSeOQdLCS8zfbAd6gqsZPWStG'

In [102]:
# Gives list of stop words and puts in variable SW 
def installnlkt():
    !pip install nltk
    import nltk
    from nltk.corpus import stopwords
    nltk.download('stopwords')
    SW = list(set(stopwords.words('english')))
    return SW


In [103]:
# Tells if client key is incorrect 
if not client_secret or not client_key:
    print("You need to fill in client_key and client_secret. See comments in the code around line 8-14")
    exit()

In [104]:
# Gets required tokens for using the API
def get_tokens():
    oauth = requests_oauthlib.OAuth1Session(client_key, client_secret=client_secret)
    request_token_url = 'https://api.twitter.com/oauth/request_token'
    fetch_response = oauth.fetch_request_token(request_token_url)
    resource_owner_key = fetch_response.get('oauth_token')
    resource_owner_secret = fetch_response.get('oauth_token_secret')

    base_authorization_url = 'https://api.twitter.com/oauth/authorize'
    authorization_url = oauth.authorization_url(base_authorization_url)
    webbrowser.open(authorization_url)    
    print("""
          Please click on the URL below, 
          click on "Authorize App", 
          and then copy the PIN and paste it below.""")
    
    print(authorization_url)
    
    verifier = input('Please input the verifier >>> ')

    oauth = requests_oauthlib.OAuth1Session(client_key,
                              client_secret=client_secret,
                              resource_owner_key=resource_owner_key,
                              resource_owner_secret=resource_owner_secret,
                              verifier=verifier)

    access_token_url = 'https://api.twitter.com/oauth/access_token'
    oauth_tokens = oauth.fetch_access_token(access_token_url)
    resource_owner_key = oauth_tokens.get('oauth_token')
    resource_owner_secret = oauth_tokens.get('oauth_token_secret')
    return (client_key, client_secret, resource_owner_key, resource_owner_secret, verifier)

# Checks to see if there is a creds.txt file 

try:
    f = open("creds.txt", 'r')
    (client_key, client_secret, resource_owner_key, resource_owner_secret, verifier) = json.loads(f.read())
    f.close()
except:
    tokens = get_tokens()
    f = open("creds.txt", 'w')
    f.write(json.dumps(tokens))
    f.close()
    (client_key, client_secret, resource_owner_key, resource_owner_secret, verifier) = tokens


In [105]:
# Performs request to use twitter data 
protected_url = 'https://api.twitter.com/1.1/account/settings.json'
oauth = requests_oauthlib.OAuth1Session(client_key,
                        client_secret=client_secret,
                        resource_owner_key=resource_owner_key,
                        resource_owner_secret=resource_owner_secret)

In [119]:
# Requests any hashtag from user, pages, and puts into 'res' 


userinput = input('Enter hashtag that you wish to have an Itunes playlist created for (Without the #) : ')
hold = userinput.lower()
hashtag = '#{}'.format(hold)


r = oauth.get("https://api.twitter.com/1.1/search/tweets.json", params = 
                                                                  {'q': hashtag,
                                                                   'lang': 'en',
                                                                   'count' : 100})

res = r.json()

new_params = {'q' : hashtag,
              'count' : 100}
new_params['max_id'] = (res['search_metadata']['max_id']) - 1
r2 = oauth.get("https://api.twitter.com/1.1/search/tweets.json", params = new_params) 
rest = r2.json()['statuses']

res['statuses'] += rest
dict_of_all_tweets = res


Enter hashtag that you wish to have an Itunes playlist created for (Without the #) : lol


In [120]:
r = oauth.get("https://api.twitter.com/1.1/search/tweets.json", params = 
                                                                  {'q': hashtag,
                                                                   'lang': 'en',
                                                                   'count' : 100})
res = r.json()
new_params = {'q' : hashtag,
              'count' : 100}
new_params['max_id'] = (res['search_metadata']['max_id']) - 1
r2 = oauth.get("https://api.twitter.com/1.1/search/tweets.json", params = new_params) 
rest = r2.json()['statuses']

res['statuses'] += rest
dict_of_all_tweets = res

if not dict_of_all_tweets:
    print('Nothing came up for this search querey, please try again')


In [121]:
# Writes all the data from the 200 tweets to 'nested.txt' to look at 
f = open('nested.txt', 'w')
f.write(pretty(dict_of_all_tweets))
f.close()

In [122]:
#definition of class Tweet 
class Tweet():
    def __init__(self, dic = {}):
        if 'text' in dic:
            self.text = dic['text']
        else:
            self.text = ''
        self.favorites = dic['favorite_count']
        self.retweets = dic['retweet_count']
    def __str__(self):
        tweet = "This body of this tweet says: {} \n".format(self.text)
        stats = "This tweet has {} retweets and {} favorites".format(self.retweets, self.favorites)
        return tweet + stats
    def stopwords(self, stopwds):
        updated = [x for x in self.text.split() if x.lower() not in SW]
        return updated
    
        
        
    

In [123]:
# creates LIST of instances of class tweet 
SW = installnlkt()

list_of_tweets = []
iterator = 0
for x in dict_of_all_tweets['statuses']: 
    y = Tweet(x)
    if iterator == 0:
        savedTweet = y 
    z = y.stopwords(SW)
    list_of_tweets.append(z)
    iterator += 1



You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [124]:
# Using those instances and their capabilities, find the most common word that is at 
#           least five letters long and is not a “stopword”, across all of the tweets.

common_words = {}
for x in list_of_tweets:
    for y in x:
        g = y.lower()
        z = g.strip('!@#$%^&*_+=-|\"/:;<,>.?/~`')
        # Eliminated the inputed word from dictionary as it would otherwise always be the most popular 
        if len(z) >= 5 and 'https' not in z and z != userinput:
            if z not in common_words:
                common_words[z] = 0
            common_words[z] += 1

sorted_words = {}
for a in sorted(common_words, key = lambda j: common_words[j], reverse = True):
    sorted_words[a] = common_words[a]
    
most_common = list(sorted_words)[0]



In [125]:
#gets info from Itunes
itunes = requests.get('https://itunes.apple.com/search?parameterkeyvalue', params = {
                     'term' : most_common,
                     'country' : 'US',
                     'media' : 'music'})
songs = itunes.json()

In [126]:
#definition of class Song
class Song():
    def __init__(self, dic = {}):
        self.title = dic['trackName']
        self.length = dic['trackTimeMillis']
        self.artist = dic['artistName']
        if 'collectionName' in dic:
            self.album = dic['collectionName']
        else:
            self.album = ''
        if 'primaryGenreName' in dic:
            self.genre = dic['primaryGenreName']
        else:
            self.genre = ''
    def __str__(self):
        z = "Song Title: {}, Song Duration: {}".format(self.title, self.converttominutes())
        return z
    def converttominutes(self):
        totsec = int(self.length / 1000)
        mins = int(totsec // 60)
        sec = int(totsec % 60)
        if sec >= 0 and sec <= 9:
            sec = '0' + str(sec)
        return '{}:{}'.format(mins, sec)

        
        


In [127]:
#creates list of Song() instances and sorts by length 
list_of_songs = [Song(x) for x in songs['results']]
sorted_songs = []
it = 0
for w in sorted(list_of_songs, reverse = True, key = lambda y: int(y.length)):
    sorted_songs.append(w)
    if it == 0:
        savedSong = w
    it += 1


    
    

In [128]:
#creates CSV file
def wsonginfo(L):
    outfile = open('songinfo.csv', 'w')
    outfile.write('Song Title, Artist, Length, Album, Genre\n')
    for t in L:
        outfile.write('{}, {}, {}, {}, {}\n'.format(t.title, t.artist, t.converttominutes(), t.album, t.genre))
    outfile.close()
    return None


In [129]:
# Summarizes what happened 
def summary(tweet, song):
    print('The program has finished running.')
    print('The most popular word found on Twitter using your hashtag was "{}"'.format(most_common))
    print('One of the Tweets you requested can be found below: \n')
    print(tweet.__str__())
    print('\nThe first song found when searching that word in Itunes can be found below: \n')
    print(song.__str__())
    print('\nThank you for using this program.')
    return None 


In [130]:
#writes final product and gives summary 
wsonginfo(sorted_songs)
summary(savedTweet, savedSong)

The program has finished running.
The most popular word found on Twitter using your hashtag was "funny"
One of the Tweets you requested can be found below: 

This body of this tweet says: @charlieventuri @ManCity I was gutted buddy...But today we won the league thanks to a class performance by… https://t.co/kyb3vlDvAQ 
This tweet has 0 retweets and 0 favorites

The first song found when searching that word in Itunes can be found below: 

Song Title: Road Manager, Song Duration: 7:36

Thank you for using this program.
